In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/curtissmith/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [3]:
# Visit the Beer Advocate Top 250 page
url = 'https://www.beeradvocate.com/beer/styles/'
browser.visit(url)

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
styles_home = soup(html, 'html.parser')

In [5]:
# Find all anchor tags
all_hrefs = styles_home.find_all('a')

In [20]:
styles_hrefs = []
error_hrefs = ['<a href="/beer/styles/">Beer Styles</a>',
              '<a href="/beer/styles/#XenForo"><i class="uix_icon uix_icon-jumpToTop"></i> <span class="uix_hide">Top</span></a>',
              '<a href="/beer/top-styles/">Beers by Style</a>',
              '<a href="/beer/top-styles/">Styles</a>']
for item in all_hrefs:
    if "/beer/styles/" in str(item):
        if str(item) not in error_hrefs:
            styles_hrefs.append(item.get('href'))

In [38]:
start_time = time.time()

count = 0
num_skipped = 0
errors = []
beer_info = []
mininum_num_reviews = 100


browser.visit(url)
styles_home = soup(html, 'html.parser')


#iterate through style links
for style_link in styles_hrefs:
    print(f'Style Link: {style_link}')
    
    try:
        
        # Setting url variable to navigate back to
        style_url = f"https://www.beeradvocate.com{style_link}"
        
        # Reseting num_reviews_over
        num_reviews_under = False
        
        next_page_links = [style_link,
                f"{style_link}?sort=revsD&start=50",]
        
        for page in next_page_links:
            
            # If the number of reviews is under the limit, go to next style
            if num_reviews_under == True:
                browser.visit(url)
                styles_home = soup(html, 'html.parser')
                continue
        
            # navigate to link
            browser.click_link_by_href(page)

            # Parse the beer html with soup
            html = browser.html
            style_profile = soup(html, 'html.parser')

            # find all a tags
            all_a = style_profile.find_all('a')

            # select only relevant link (those with "beer/profile" in link)
            links = [x for x in all_a if '/beer/profile/' in str(x)]

            # iterate and get links to beers - even indices are links to beers, odd are links to brewery
            beer_a_tags = []
            for index, item in enumerate(links):
                if index % 2 == 0:
                    beer_a_tags.append(item)

            # get href url for navigating to beer webpage
            beer_href = [x.get('href') for x in beer_a_tags]

            #iterate through links
            for link in beer_href:
                
#                 print(f'href {link}')
                
#                 if num_reviews_under == True:
#                     browser.visit(url)
#                     styles_home = soup(html, 'html.parser')
#                     continue
                
                count += 1
                if count % 50 == 0:
                    length = (time.time() - start_time)
                    print(f'Elapsed Time: {length} seconds / {length/60} minutes')
                    print(f'Parsed {count} beers')
                    print(f'Added {len(beer_info)} beers with >= {mininum_num_reviews} reviews')

                # navigate to link
                browser.click_link_by_href(link)

                # Parse the beer html with soup
                html = browser.html
                beer_soup = soup(html, 'html.parser')
#                 print(browser.url)
#                 print(beer_soup.find_all('a'))
                
                try:

                    reviews = []

                    spans = beer_soup.find_all('span', class_='Tooltip')
                    num_reviews = spans[10].text
                    num_reviews = int(num_reviews.replace(",", ""))

                    # Only have beers with > 100 reviews
                    if num_reviews < mininum_num_reviews:
    #                     print(num_reviews)
                        num_skipped += 1
#                         num_reviews_under = True

                        browser.visit(style_url)

                        # Parse the beer html with soup
                        html = browser.html
                        style_profile = soup(html, 'html.parser')

                        continue

                    sorting_links = [f"{link}?sort=top",
                                    f"{link}?view=beer&sort=top&start=25",
                                    f"{link}?view=beer&sort=top&start=50",
                                    f"{link}?view=beer&sort=top&start=100"]


                    for item in sorting_links:

#                         print(browser.url)
                        
                        browser.click_link_by_href(item)

                        html = browser.html
                        beer_soup = soup(html, 'html.parser')

                        # Get beer and brewery
                        header = beer_soup.find('div', class_='titleBar')
                        brewery = header.find("span").text
                        beer_name = header.text.replace(brewery, "")
                        beer_name = beer_name.replace('\n', "")

                        # get ABV and rating
                        spans = beer_soup.find_all('span', class_='Tooltip')
                        abv = spans[2].text
                        rating = spans[6].text

                        # get style
                        a = beer_soup.find_all('a', class_='Tooltip')
                        style = a[4].text

                        # get reviews
                        review_div = beer_soup.find_all('div', {"id": "rating_fullview_content_2"})
                        for item in review_div:
                            reviews.append(item.text.split("%")[1])

                    # create dictionary and add to list
                    beer_info.append({"beer_name": beer_name,
                                      "beer_style": style,
                                      "brewery": brewery,
                                      "abv": abv,
                                      "rating": rating,
                                      "reviews": reviews})

                    #print(len(reviews))

                    browser.visit(style_url)

                    # Parse the beer html with soup
                    html = browser.html
                    style_profile = soup(html, 'html.parser')                       
            
                except Exception as exception:
                    print(exception)
                    errors.append(beer_name)
                    print(f'Current Error Count: {len(errors)}')


                    if browser.url == 'https://www.google.com/':
                        break

                    browser.visit(style_url)

                    # Parse the beer html with soup
                    html = browser.html
                    style_profile = soup(html, 'html.parser') 
                    
    except Exception as e:
        print(e)

    if browser.url == 'https://www.google.com/':
        break

    browser.visit(url)

    # Parse the beer html with soup
    browser.visit(url)
    styles_home = soup(html, 'html.parser')
    
length = (time.time() - start_time)
print(f'Script ran in {length} seconds / {length/60} minutes')
print(f'Obtained {len(beer_info)} beer records')
print(f'Skipped {num_skipped} beers')
print(f'There were {len(errors)} errors')

Style Link: /beer/styles/35/
'NoneType' object has no attribute 'find'
Current Error Count: 1
no elements could be found with link by href "/beer/styles/35/?sort=revsD&start=50"
Script ran in 18.703238248825073 seconds / 0.3117206374804179 minutes
Obtained 0 beer records
Skipped 0 beers
There were 1 errors


In [ ]:
"/beer/profile/39/131/?sort=top"